In [1]:
# imports
import folium
from haversine import haversine, Unit
from pprint import pprint
import numpy as np
import time as t
import pandas as pd
from datetime import datetime, timedelta
from decimal import Decimal, ROUND_UP
import osmnx as ox
import networkx as nx

ox.config(log_console=False, use_cache=True)

In [2]:
# Khi - HQ
Khi_latlng = (24.853803, 67.035395)

# Saleh Pat
slp_latlng = (27.519, 69.044)

# Hyd - Cantt
hyd_latlang = (25.399066, 68.359492)

# Bdn - Cantt
bdn_latlang = (24.657308, 68.859181)


In [3]:
# location where you want to find your route
place     = 'Sindh, Pakistan'
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'length'        # 'length','time'

# Get all nodes in a certain area which are permissible for driving
graph = ox.graph_from_place(place, network_type = mode, simplify = False)

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [23]:
def zoom_path(path_list):
    
    interpol_path_list = []
    
    for i in range(len(path_list)-1):
        
        interpol_val_x = (path_list[i][0] + path_list[i+1][0])/2
        interpol_val_y = (path_list[i][1] + path_list[i+1][1])/2
        interpol_val = (interpol_val_x, interpol_val_y)
        
        interpol_path_list.append(path_list[i])
        interpol_path_list.append(interpol_val)
        
    print('len of original data: ', len(path_list))
    print('len of interpolated data: ', len(interpol_path_list))
    
    return interpol_path_list

In [24]:
def get_path_coordinates(cvy_start, cvy_dest):
    
    path_list = []
    if cvy_start == 'Karachi':
        # Khi - HQ
        start_latlng = Khi_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
        
    if cvy_start == 'Hyderabad':
        # Khi - HQ
        start_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
            
    if cvy_start == 'Badin':
        # Khi - HQ
        start_latlng = bdn_latlang
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
    if cvy_start == 'Saleh Pat':
        # Khi - HQ
        start_latlng = slp_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng

    
    # find the nearest node to the start location
    orig_node = ox.get_nearest_node(graph, start_latlng)

    # find the nearest node to the end location
    dest_node = ox.get_nearest_node(graph, end_latlng)
    
    #  find the shortest path
    shortest_route = nx.shortest_path(graph,
                                      orig_node,
                                      dest_node,
                                      weight=optimizer)
    
    for shortlisted_node in shortest_route:
        path_list.append((graph.nodes[shortlisted_node]["y"], 
                          graph.nodes[shortlisted_node]["x"]))
    
    interpol_path_list = zoom_path(path_list)
    
    return interpol_path_list

In [25]:
def get_input_distance(start_time, user_time, speed):
    
    t1 = datetime.strptime(str(start_time)[0:2] + ':' + str(start_time)[2:4], '%H:%M')
    t2 = datetime.strptime(str(user_time)[0:2] + ':' + str(user_time)[2:4], '%H:%M')
    td = t2 - t1
    time_spent = td.total_seconds()/3600
    dist_now = speed*time_spent
    
    return dist_now


In [26]:
def get_dist_coordinates(coordinates):
    
    cumulative_dist = 0
    dist_list = []
#     print(coordinates)
    
    for index in range(len(coordinates)):
        if index:
            dist_one = haversine(coordinates[index-1], coordinates[index])
            cumulative_dist += dist_one
            dist_list.append(cumulative_dist)
        else:
            dist_one = 0
            dist_list.append(dist_one)
    
    return dist_list


In [27]:
def get_relative_distance(user_distance, dist_coordinates):
    
    relative_dist = []
#     print(user_distance)
    
    for dist in dist_coordinates:
        
        temp = abs(dist - user_distance)
        relative_dist.append(temp)
        
    return relative_dist

In [28]:
def create_path(coordinates_cleaned, cvy_start, cvy_dest, cvy_color, folium_map):
# expects a list of tuples containing path coordinates ... [(1,1),(2,2),(3,3)...]
    
    path = coordinates_cleaned
    
    folium.PolyLine(path,
                color= cvy_color,
                weight=3,
                opacity=0.7).add_to(folium_map)

In [29]:
"""Read data from excel file"""

# lst of column names which needs to be string
lst_str_cols = ['Start Time']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}

cvy_inputs = pd.read_excel('Inputs.xlsx', dtype=dict_dtypes)
cvy_inputs['Coordinates'] = cvy_inputs['Coordinates'].astype(object)
cvy_inputs['Distance'] = cvy_inputs['Distance'].astype(object)
cvy_inputs['Relative Distance'] = cvy_inputs['Relative Distance'].astype(object)
start_time = cvy_inputs['Start Time'].min()

for index, rows in cvy_inputs.iterrows():
    
    # get coordinates based on route and append to dataframe as new column 'Coordinates'
    path = get_path_coordinates(rows['Start'], rows['Destination'])
    # insert list of coordinates at a single index in dataframe
    cvy_inputs.at[index, 'Coordinates'] = path

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


len of original data:  1236
len of interpolated data:  2470


C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


len of original data:  2134
len of interpolated data:  4266


C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


len of original data:  3248
len of interpolated data:  6494


C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


len of original data:  780
len of interpolated data:  1558


In [31]:
basic_map = folium.Map(location= (26.277998, 68.288195),
                    zoom_start=7)

folium.TileLayer('Stamen Terrain').add_to(basic_map)
folium.TileLayer('Stamen Toner').add_to(basic_map)
folium.TileLayer('Stamen Water Color').add_to(basic_map)
folium.TileLayer('cartodbpositron').add_to(basic_map)
folium.TileLayer('cartodbdark_matter').add_to(basic_map)
folium.LayerControl().add_to(basic_map)

user_time = str(input('Enter Time for Activity XXXX Hrs: '))

user_dist = get_input_distance(start_time, user_time, cvy_inputs['Speed'])

# calculate distances between coordinates

for index, rows in cvy_inputs.iterrows():
    # get distances based on coordinates and append to dataframe as new column 'Distance'
    dist = get_dist_coordinates(rows['Coordinates'])
    # insert list of distances at a single index in dataframe
    cvy_inputs.at[index, 'Distance'] = dist

# find shortest distance of user_distance - rows['Distance']

for index, rows in cvy_inputs.iterrows():
    relative_dist = get_relative_distance(user_dist[index], rows['Distance'])
    cvy_inputs.at[index, 'Relative Distance'] = relative_dist
    

for index, rows in cvy_inputs.iterrows():
    
    num_vehicles = range(rows['Order'])
    mid_pt = round(len(num_vehicles)/2)
    
    prev_index = 0
    
    for vehicle in num_vehicles:
        
        index_min = np.argmin(rows['Relative Distance'])
        coordinates_min = rows['Coordinates'][index_min - vehicle]
        
        if vehicle == mid_pt:
            folium.Circle(location= coordinates_min,
                                radius=100*len(num_vehicles)/2,
                                fill_color= rows['Color']).add_to(basic_map)
        
        popup_text = str(rows['Name']) + ' Current Location: ' + str(user_dist[index]) + " KM"


        folium.Marker(location= coordinates_min, 
                      popup= popup_text,
                      icon= folium.Icon(color=rows['Color'], 
                                        icon_color='white', 
                                        icon = 'fa-truck', 
                                        prefix='fa')
                     ).add_to(basic_map)
    
        prev_index = coordinates_min
        
        
#         folium.RegularPolygonMarker(location= coordinates_min 
#                                 , fill_color= rows['Color']
#                                 , number_of_sides=3
#                                 , radius=15
#                                 , rotation=45).add_to(basic_map)
        
        
        create_path(rows['Coordinates'], rows['Start'], rows['Destination'], rows['Color'], basic_map)
    

basic_map

Enter Time for Activity XXXX Hrs: 1030
